In [1]:
from utils import GeneSeg
import csv,pickle,random,json
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf

vec_dir="/home/admin/xssdetection/file/word2vec.pickle"
pre_datas_train="/home/admin/xssdetection/file/pre_datas_train.csv"
pre_datas_test="/home/admin/xssdetection/file/pre_datas_test.csv"
process_datas_dir="/home/admin/xssdetection/file/process_datas.pickle"

/home/admin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def pre_process():
    with open(vec_dir,"rb") as f :
        word2vec=pickle.load(f)
        dictionary=word2vec["dictionary"]
        reverse_dictionary=word2vec["reverse_dictionary"]
        embeddings=word2vec["embeddings"]
    xssed_data=[]
    normal_data=[]
    with open("/home/admin/xssdetection/data/xssed.csv","r",encoding="utf-8") as f:
        reader = csv.DictReader(f, fieldnames=["payload"])
        for row in reader:
            payload=row["payload"]
            word=GeneSeg(payload)
            xssed_data.append(word)
    with open("/home/admin/xssdetection/data/normal_examples.csv","r",encoding="utf-8") as f:
        reader=csv.reader(f)
        reader = csv.DictReader(f, fieldnames=["payload"])
        for row in reader:
            payload=row["payload"]
            word=GeneSeg(payload)
            normal_data.append(word)
    xssed_num=len(xssed_data)
    normal_num=len(normal_data)
    xssed_labels=[1]*xssed_num
    normal_labels=[0]*normal_num
    datas=xssed_data+normal_data
    labels=xssed_labels+normal_labels
    labels=to_categorical(labels)
    def to_index(data):
        d_index=[]
        for word in data:
            if word in dictionary.keys():
                d_index.append(dictionary[word])
            else:
                d_index.append(dictionary["UNK"])
        return d_index
    datas_index=[to_index(data) for data in datas]
    #print(datas_index[0:100])
    datas_index=pad_sequences(datas_index,value=-1)
    print(type(datas_index),type(datas_index[0]))
    rand=random.sample(range(len(datas_index)),len(datas_index))
    datas=[datas_index[index] for index in rand]
    labels=[labels[index] for index in rand]
    train_datas,test_datas,train_labels,test_labels=train_test_split(datas,labels,test_size=0.3)
    print(type(train_datas),type(train_datas[0]))
    print(np.shape(train_datas))
    print(embeddings["UNK"].shape)
    train_size=len(train_labels)
    test_size=len(test_labels)
    input_num=len(train_datas[0])
    dims_num = embeddings["UNK"].shape[0]
    word2vec["train_size"]=train_size
    word2vec["test_size"]=test_size
    word2vec["input_num"]=input_num
    word2vec["dims_num"]=dims_num
    with open(vec_dir,"wb") as f :
        pickle.dump(word2vec,f)
    print("Saved word2vec to:",vec_dir)
    print("Write trian datas to:",pre_datas_train)
    with open(pre_datas_train,"w") as f:
        for i in range(train_size):
            data_line=str(train_datas[i].tolist())+"|"+str(train_labels[i].tolist())+"\n"
            f.write(data_line)
    print("Write test datas to:",pre_datas_test)
    with open(pre_datas_test,"w") as f:
        for i in range(test_size):
            data_line=str(test_datas[i].tolist())+"|"+str(test_labels[i].tolist())+"\n"
            f.write(data_line)
    print("Write datas over!")


In [3]:
pre_process()

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'list'> <class 'numpy.ndarray'>
(168536, 532)
(128,)
Saved word2vec to: /home/admin/xssdetection/file/word2vec.pickle
Write trian datas to: /home/admin/xssdetection/file/pre_datas_train.csv
Write test datas to: /home/admin/xssdetection/file/pre_datas_test.csv
Write datas over!


In [4]:
def data_generator(data_dir):
    reader = tf.TextLineReader()
    queue = tf.train.string_input_producer([data_dir])
    _, value = reader.read(queue)
    coord = tf.train.Coordinator()
    sess = tf.Session()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    while True:
        v = sess.run(value)
        [data, label] = v.split(b"|")
        data = np.array(json.loads(data.decode("utf-8")))
        label = np.array(json.loads(label.decode("utf-8")))
        yield (data, label)
    coord.request_stop()
    coord.join(threads)
    sess.close()


In [5]:
generator=data_generator(pre_datas_train)

In [6]:
next(generator)

(array([ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1

In [7]:
def batch_generator(datas_dir,datas_size,batch_size,embeddings,reverse_dictionary,train=True):
    batch_data = []
    batch_label = []
    generator=data_generator(datas_dir)
    n=0
    while True:
        for i in range(batch_size):
            data,label=next(generator)
            data_embed = []
            for d in data:
                if d != -1:
                    data_embed.append(embeddings[reverse_dictionary[d]])
                else:
                    data_embed.append([0.0] * len(embeddings["UNK"]))
            batch_data.append(data_embed)
            batch_label.append(label)
            n+=1
            if not train and n==datas_size:
                break
        if not train and n == datas_size:
            yield (np.array(batch_data), np.array(batch_label))
            break
        else:
            yield (np.array(batch_data),np.array(batch_label))
            batch_data = []
            batch_label = []
def build_dataset(batch_size):
    with open(vec_dir, "rb") as f:
        word2vec = pickle.load(f)
    embeddings = word2vec["embeddings"]
    reverse_dictionary = word2vec["reverse_dictionary"]
    train_size=word2vec["train_size"]
    test_size=word2vec["test_size"]
    dims_num = word2vec["dims_num"]
    input_num =word2vec["input_num"]
    train_generator = batch_generator(pre_datas_train,train_size,batch_size,embeddings,reverse_dictionary)
    test_generator = batch_generator(pre_datas_test,test_size,batch_size,embeddings,reverse_dictionary,train=False)
    return train_generator,test_generator,train_size,test_size,input_num,dims_num

train_generator, test_generator, train_size, test_size, input_num, dims_num=build_dataset(500)

In [8]:
len(next(train_generator)[0]),train_size,test_size,input_num,dims_num

(500, 168536, 72230, 532, 128)

In [9]:
import time
from keras.models import Sequential
from keras.layers import Dense,InputLayer,Dropout,LSTM
from keras.optimizers import Adam
batch_size=500
epochs_num=1
process_datas_dir="/home/admin/xssdetection/file/process_datas.pickle"
model_dir="/home/admin/xssdetection/file/LSTM_model"
def train(train_generator,train_size,input_num,dims_num):
    print("Start Train Job! ")
    start=time.time()
    ######inputs=InputLayer(input_shape=(input_num,dims_num),batch_size=batch_size)###why add the line cause model sava failed?
    layer1=LSTM(16,input_shape=(input_num,dims_num),batch_size=batch_size)
    output=Dense(2,activation="softmax",name="Output")
    model=Sequential()
    #model.add(inputs)
    model.add(layer1)
    model.add(output)
    model.summary()
    model.compile(optimizer='rmsprop',loss="categorical_crossentropy",metrics=["accuracy"])
    model.fit_generator(train_generator,steps_per_epoch=train_size//batch_size,epochs=epochs_num)
#    model.fit_generator(train_generator, steps_per_epoch=5, epochs=5, callbacks=[call])
    end=time.time()
    print("Over train job in %f s"%(end-start))
    return model


In [10]:
model=train(train_generator,train_size,input_num,dims_num)

Start Train Job! 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (500, 16)                 9280      
_________________________________________________________________
Output (Dense)               (500, 2)                  34        
Total params: 9,314
Trainable params: 9,314
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1
337/337 [==============================] - 1359s 4s/step - loss: 0.0402 - acc: 0.9901
Over train job in 1361.460702 s


In [18]:
model.predict_generator(test_generator,steps=1).shape

(500, 2)

In [12]:
#from keras.models import model_from_json
filename = './file/xss_lstm.h5'
model.save(filename)
#model.save_weights(filename)
#model.load_weights(filename,by_name=True)
#json_string = model.to_json()
#model = model_from_json(json_string)


In [13]:
from keras.models import load_model
model=load_model('./file/xss_lstm.h5')
model.predict_generator(test_generator,steps=1)

array([[9.9938452e-01, 6.1541318e-04],
       [9.9871635e-01, 1.2836576e-03],
       [9.9899739e-01, 1.0026500e-03],
       ...,
       [4.1994863e-04, 9.9958009e-01],
       [9.9857724e-01, 1.4227253e-03],
       [9.9821019e-01, 1.7897724e-03]], dtype=float32)